In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [128]:
data_train = pd.read_csv('train.csv')
data_train_len = data_train.shape[0]
data_test = pd.read_csv('test.csv')
y = data_train.pop('Attrition')
data = pd.concat([data_train, data_test])

data = data.drop(columns=['EmployeeID', 'StandardHours', 'Over18', 'EmployeeCount'])
data.replace({'JobRole':{'Admin': 'Administrative'}}, inplace=True)
# for colname in data.select_dtypes("O"):
#     data[colname], _ = data[colname].factorize()
data.replace({'Education':{1:'Below_college', 2:'College', 3:'Bachelor', 4:'Masters', 5:'Doctor'}}, inplace=True)
data = pd.get_dummies(data)
data = data.drop(columns=['Gender_Female','OverTime_No'])

train = data.iloc[:data_train_len, :]
test = data.iloc[data_train_len:, :]

In [129]:
train.shape, test.shape

((1340, 48), (336, 48))

In [130]:
def y_enco(y):
    if y == 'No':
        return 0
    elif y == 'Yes':
        return 1
    else:
        pass

y_encoder = y.apply(y_enco)

In [131]:
from imblearn.over_sampling import ADASYN 
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
# Data Normalization

# ada = ADASYN(random_state=42)
# X_res, y_res = ada.fit_resample(X_norm, y_encorder)
# X_res.shape

X_train, X_test, y_train, y_test = train_test_split(train, y_encoder, test_size=0.1, random_state=42)


scaler = MinMaxScaler(feature_range = (0,1))
scaler.fit(X_train)
X_train_norm = scaler.transform(X_train)


ada = ADASYN(random_state=42)
X_res, y_res = ada.fit_resample(X_train_norm, y_train)
X_res.shape, y_res.shape

((2133, 48), (2133,))

In [132]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier



# Defining parameters for hyper-parameter tuning
params = {
        'n_estimators': [int(x) for x in np.linspace(start = 800, stop = 1200, num = 5)],
        'learning_rate': [0.01, 0.1, 1.0],

}

# Initializing Grid Search with Logistic Regression and keeping roc_auc as the performance metrics!
grid_search = GridSearchCV(estimator = AdaBoostClassifier(),
                           param_grid=params,
                           cv = 10,
                           n_jobs=-1,
                           verbose=0,
                           scoring="f1",
                           return_train_score=True)

# Training
grid_search.fit(X_res, y_res)

# Best Performing Parameter
print('='*20)
print("best params: " + str(grid_search.best_estimator_))
print("best params: " + str(grid_search.best_params_))
print('best score:', grid_search.best_score_)
print('='*20)

best params: AdaBoostClassifier(learning_rate=0.1, n_estimators=1100)
best params: {'learning_rate': 0.1, 'n_estimators': 1100}
best score: 0.9554286436015518


In [133]:
from sklearn.metrics import f1_score

X_val_norm = scaler.transform(X_test)

y_pred = grid_search.predict(X_val_norm)

f1_score(y_test, y_pred)

0.6428571428571429

In [120]:
X_test_norm = scaler.transform(test)

y_pred = grid_search.predict(X_test_norm)

id = range(0,len(y_pred))
y_test = pd.DataFrame()
y_test['Id'] = id
y_test['Predicted'] = y_pred
y_test.to_csv('pred_ada_ad.csv', index = 0)